In [5]:
from tqdm.auto import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets
from typing import Optional
import os
import csv
import nest_asyncio
nest_asyncio.apply()

In [6]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") # or load from data/test.csv
print(eval_dataset)

Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
2025-08-13 16:41:54,477 - WARNING - Using the latest cached version of the dataset since m-ric/huggingface_doc_qa_eval couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).
2025-08-13 16:41:54,479 - WARNING - Found the latest cached dataset configuration 'default' at /home/minhthuy/.cache/huggingface/datasets/m-ric___huggingface_doc_qa_eval/default/0.0.0/5f70aa9a1e2430f528ac3f27f01f0ba8719c0704 (last modified on Mon Aug 11 14:41:56 2025).


Dataset({
    features: ['context', 'question', 'answer', 'source_doc', 'standalone_score', 'standalone_eval', 'relatedness_score', 'relatedness_eval', 'relevance_score', 'relevance_eval'],
    num_rows: 65
})


In [7]:
OUTPUT_FILE = "data/rag_results_k=20_fetch=40_top=3.csv"

def load_existing_results(file_path):
    dataset = []
    existing_questions = set()
    if os.path.exists(file_path):
        with open(file_path, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_questions.add(row["user_input"])
                # parse retrieved_contexts lại từ JSON
                row["retrieved_contexts"] = json.loads(row["retrieved_contexts"])
                dataset.append(row)
    return existing_questions, dataset


def append_result_to_csv(file_path, row, fieldnames):
    file_exists = os.path.exists(file_path)
    with open(file_path, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)

def run_rag_tests(
    eval_dataset: datasets.Dataset,
    verbose: Optional[bool] = False
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    fieldnames = ["user_input", "retrieved_contexts", "response", "reference"]

    processed_questions, dataset = load_existing_results(OUTPUT_FILE)
    print(f"Đã có {len(processed_questions)} câu hỏi xử lý trước đó, sẽ bỏ qua chúng.")


    for i, example in enumerate(tqdm(eval_dataset)):
        question = example["question"]

        if question in processed_questions:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4()),
            "chat_history": []
        }

        try:
            relevant_docs, answer = asyncio.run(call_chat_once(payload))
        except Exception as e:
            print(f"Error from question: '{question}': {e}")

        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')

        row = {
            "user_input": question,
            "retrieved_contexts": json.dumps(
                [doc['kwargs'].get('page_content') for doc in relevant_docs],
                ensure_ascii=False
            ),
            "response": answer,
            "reference": example['answer']
        }
        append_result_to_csv(OUTPUT_FILE, row, fieldnames)
        dataset.append({
            "user_input": question,
            "retrieved_contexts": json.loads(row["retrieved_contexts"]),
            "response": answer,
            "reference": example['answer']
        })
    return dataset

In [8]:
data = run_rag_tests(eval_dataset=eval_dataset)


Đã có 0 câu hỏi xử lý trước đó, sẽ bỏ qua chúng.


  2%|▏         | 1/65 [00:13<14:16, 13.38s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What architecture is the `tokenizers-linux-x64-musl` binary designed for?\n', 'type': 'human', 'id': '2da07b5e-4305-4749-906e-7195019a833d'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_pWU71Mzvryg1qnSeK0CW6jwR', 'function': {'arguments': '{"query":"tokenizers-linux-x64-musl architecture"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--4f3d3724-67ec-4ed1-8aa5-7c16b659e207-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'tokenizers-linux-x64-musl architecture'}, 'id': 'call_pWU71Mzvryg1qnSeK0CW6jwR', 'type': 'tool_call'}], 'usage_metadata': {'input_tok

2025-08-13 16:42:22,886 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
  3%|▎         | 2/65 [00:28<15:03, 14.33s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the purpose of the BLIP-Diffusion model?\n', 'type': 'human', 'id': '09c945f9-8a47-4fa3-bfc1-7d5d1f82c54f'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_Q7v5M8Ro1d8Hc5lbyy6viuQN', 'function': {'arguments': '{"query":"BLIP-Diffusion model purpose"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--93e503c4-6c95-4727-b8c9-6c6a02851fe7-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'BLIP-Diffusion model purpose'}, 'id': 'call_Q7v5M8Ro1d8Hc5lbyy6viuQN', 'type': 'tool_call'}], 'usage_metadata': {'input_tokens': 361, 'output_tokens': 22, 'total_tokens

...

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the name of the open-source library created by Hugging Face to simplify Transformer acceleration?\n', 'type': 'human', 'id': '7a410303-7a0b-4791-a545-593f903cf259'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_jbId4p8u2gUlrxBKgsGO0V3I', 'function': {'arguments': '{"query":"open-source library created by Hugging Face to simplify Transformer acceleration"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--a2c36206-67e1-424b-ba81-5ac65de7c696-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'open-source library created by Hugging Face to simplify Tra

2025-08-13 16:56:50,317 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
 98%|█████████▊| 64/65 [14:55<00:11, 11.66s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What parameter is used to ensure that elements in a row have the same height in Gradio?\n', 'type': 'human', 'id': '82a47c74-2459-4421-9e3b-934e15abd710'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_mB2tCMierifKLE2UD8UIJ7YT', 'function': {'arguments': '{"query":"Gradio same height elements in a row parameter"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--5b04eada-5252-4ff3-8f92-729babd4323a-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'Gradio same height elements in a row parameter'}, 'id': 'call_mB2tCMierifKLE2UD8UIJ7YT', 'type': 'tool_call'}],

2025-08-13 16:57:04,887 - INFO - HTTP Request: POST http://localhost:8001/api/v1/chat-evaluation "HTTP/1.1 200 OK"
100%|██████████| 65/65 [15:10<00:00, 14.01s/it]

{'messages': [{'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'HumanMessage'], 'kwargs': {'content': 'What is the command to install the latest version of Optimum with OpenVINO support?\n', 'type': 'human', 'id': '6b539aca-0d17-416f-a842-7d77dec7ddb3'}}, {'lc': 1, 'type': 'constructor', 'id': ['langchain', 'schema', 'messages', 'AIMessage'], 'kwargs': {'content': '', 'additional_kwargs': {'tool_calls': [{'index': 0, 'id': 'call_FURVadccuoXMHu7SicJLM5kF', 'function': {'arguments': '{"query":"install latest version of Optimum with OpenVINO support"}', 'name': 'data_retriever'}, 'type': 'function'}]}, 'response_metadata': {'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c'}, 'type': 'ai', 'id': 'run--3fe81973-80cf-4817-af6b-fb74b911ab44-0', 'tool_calls': [{'name': 'data_retriever', 'args': {'query': 'install latest version of Optimum with OpenVINO support'}, 'id': 'call_FURVadccuoXMHu7SicJLM5kF', 'type': 

In [12]:
from ragas import EvaluationDataset
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    FactualCorrectness
)
evaluation_dataset = EvaluationDataset.from_list(data)

# Khởi tạo các metric
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
]
# metrics = [FactualCorrectness()]

In [13]:
print(len(evaluation_dataset))

65


In [14]:
from app.config import configs

import os
os.environ['OPENAI_API_KEY'] = configs.OPENAI_API_KEY

result = evaluate(
    evaluation_dataset,
    metrics=metrics,
)

print(result) 

Evaluating:   0%|          | 0/260 [00:00<?, ?it/s]2025-08-13 16:59:03,450 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,465 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,478 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,487 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,491 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,522 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:03,543 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-13 16:59:04,346 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-08-13 16:59:04,932 - INFO - HTTP Request: POST

{'faithfulness': 0.6557, 'answer_relevancy': 0.8714, 'context_precision': 0.5590, 'context_recall': 0.6256}
